## Field Vis

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def create_football_field(ax=None):

    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 6.33))
    else:
        fig = None  # When passing ax, no new figure is created

    # Field outline and endzones
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1, edgecolor='r', facecolor='lightgreen', zorder=0)
    ax.add_patch(rect)

    # Endzones
    ax.add_patch(patches.Rectangle((0, 0), 10, 53.3, linewidth=0.1, facecolor='blue', alpha=0.2, zorder=0))
    ax.add_patch(patches.Rectangle((110, 0), 10, 53.3, linewidth=0.1, facecolor='blue', alpha=0.2, zorder=0))

    # Field lines every 10 yards
    for x in range(10, 120, 10):
        ax.plot([x, x], [0, 53.3], color='white', zorder=1)

    ax.set_xlim(0, 120)
    ax.set_ylim(0, 53.3)
    ax.axis('off')

    return fig, ax

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML
%matplotlib inline

def animate_play(play_data):

    frame_ids = sorted(play_data['frameId'].unique())

    clubs = play_data['club'].unique()
    team_1 = clubs[0] if 'football' not in clubs[0].lower() else clubs[1]
    team_2 = clubs[1] if team_1 != clubs[1] else clubs[2]

    fig, ax = create_football_field()

    def update(frame_id):

        ax.clear()
        create_football_field(ax)

        frame_data = play_data[play_data['frameId'] == frame_id]

        man_prob = float(frame_data['man_prob'].iloc[0])
        zone_prob = float(frame_data['zone_prob'].iloc[0])

        man_prob = round(man_prob, 2)
        zone_prob = round(zone_prob, 2)

        team_1_data = frame_data[frame_data['club'] == team_1]
        team_2_data = frame_data[frame_data['club'] == team_2]
        football_data = frame_data[frame_data['club'].str.contains('football', case=False)]

        ax.scatter(team_1_data['x_clean'], team_1_data['y_clean'], color='purple', s=50, label=team_1)
        ax.scatter(team_2_data['x_clean'], team_2_data['y_clean'], color='white', s=50, label=team_2)
        ax.scatter(football_data['x_clean'], football_data['y_clean'], color='brown', s=25, label='football')

        # Add arrows to represent direction and speed
        for i in range(len(frame_data)):
            angle_rad = np.deg2rad(frame_data.iloc[i]['dir_clean'])
            dx = frame_data.iloc[i]['s_clean'] * np.cos(angle_rad)
            dy = frame_data.iloc[i]['s_clean'] * np.sin(angle_rad)
            ax.arrow(frame_data.iloc[i]['x_clean'], frame_data.iloc[i]['y_clean'], dx, dy,
                     head_width=0.25, head_length=0.5, fc='red', ec='red')

        # Title showing the FrameId
        ax.set_title(f"Frame: {frame_id} | Man Prob: {man_prob} | Zone Prob: {zone_prob}")

    # Create the animation
    ani = animation.FuncAnimation(fig, update, frames=frame_ids, repeat=False)

    # Save the animation as a video file (optional)
    ani.save(f"play_sample.mp4", writer='ffmpeg', fps=10)

    return HTML(ani.to_jshtml())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
%matplotlib inline

def plot_tell_play(play_data, player_name):

  play_data = play_data.sort_values(by='frameId')
  player_data = play_data[play_data['displayName'] == player_name]

  x_positions = player_data['x_clean'].values
  y_positions = player_data['y_clean'].values
  frame_ids = player_data['frameId'].values

  cmap = LinearSegmentedColormap.from_list("BlueToRed", ["blue", "red"])

  normalized_frames = (frame_ids - frame_ids.min()) / (frame_ids.max() - frame_ids.min())

  fig, ax = create_football_field()

  scatter = ax.scatter(
      x_positions,
      y_positions,
      c=normalized_frames,
      cmap=cmap,
      s=10
  )

  # Add a colorbar to show the frame progression
  cbar = plt.colorbar(scatter, ax=ax, orientation="vertical", pad=0.02)
  cbar.set_label("Frame Progression (Blue -> Red)")

  # Add labels and title
  ax.set_title(f"Trajectory of {player_name} (Gradient: Early -> Late Frames)")
  plt.show()